In [97]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("nhl.csv")
cities_nhl = pd.read_html("wikipedia_data.html")[1]
cities_nhl = cities_nhl.iloc[:-1, [0, 3, 5, 6, 7, 8]]
nhl_df.drop([0, 9, 18, 26], inplace=True)
cities_nhl.drop([
    14, 15, 18, 19, 20, 21, 23, 24, 25, 27, 28, 32, 33,
    38, 40, 41, 42, 44, 45, 46, 48, 49, 50], inplace=True)


def proccess_regex(pattern, row):
    result = re.findall(pattern, row)
    if result:
        return result[0]
    return row


cities_nhl['NHL'] = cities_nhl['NHL'].apply(
    lambda x: proccess_regex('([\w\s]+)(?:\[.*\])', x)
)
nhl_df['team'] = nhl_df['team'].apply(
    lambda x: proccess_regex('[^*]+', x)
)
# Only 2018 records
nhl_df = nhl_df.head(31)
nhl_df['team_ville'] = nhl_df['team']
nhl_df['team_ville'] = nhl_df['team_ville'].map({
    'Tampa Bay Lightning': 'Tampa Bay Area',
    'Boston Bruins': 'Boston',
    'Toronto Maple Leafs': 'Toronto',
    'Florida Panthers': 'Miami–Fort Lauderdale',
    'Detroit Red Wings': 'Detroit',
    'Montreal Canadiens': 'Montreal',
    'Ottawa Senators': 'Ottawa',
    'Buffalo Sabres': 'Buffalo',
    'Washington Capitals': 'Washington, D.C.',
    'Pittsburgh Penguins': 'Pittsburgh',
    'Philadelphia Flyers': 'Philadelphia',
    'Columbus Blue Jackets': 'Columbus',
    'New Jersey Devils': 'New York City',
    'Carolina Hurricanes': 'Raleigh',
    'New York Islanders': 'New York City',
    'New York Rangers': 'New York City',
    'Nashville Predators': 'Nashville',
    'Winnipeg Jets': 'Winnipeg',
    'Minnesota Wild': 'Minneapolis–Saint Paul',
    'Colorado Avalanche': 'Denver',
    'St. Louis Blues': 'St. Louis',
    'Dallas Stars': 'Dallas–Fort Worth',
    'Chicago Blackhawks': 'Chicago',
    'Vegas Golden Knights': 'Las Vegas',
    'Anaheim Ducks': 'Los Angeles',
    'San Jose Sharks': 'San Francisco Bay Area',
    'Los Angeles Kings': 'Los Angeles',
    'Calgary Flames': 'Calgary',
    'Edmonton Oilers': 'Edmonton',
    'Vancouver Canucks': 'Vancouver',
    'Arizona Coyotes': 'Phoenix'})

df_nhl = pd.merge(nhl_df, cities_nhl, left_on="team_ville",
              right_on="Metropolitan area")

df_nhl['W'] = pd.to_numeric(df_nhl['W'])
df_nhl['L'] = pd.to_numeric(df_nhl['L'])
df_nhl['Population (2016 est.)[8]'] = pd.to_numeric(
    df_nhl['Population (2016 est.)[8]']
)

df_nhl = df_nhl[['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']]

df_nhl['W/L'] = df_nhl['W'] / (df_nhl['L'] + df_nhl['W'])

df_nhl = df_nhl.groupby('Metropolitan area').mean().reset_index()
population_by_region = df_nhl['Population (2016 est.)[8]']
win_loss_by_region = df_nhl['W/L']
stats.pearsonr(population_by_region, win_loss_by_region)[0]


0.012486162921209923

---

---

In [98]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'other', 'foo', 'hello'],
                    'value': [1, 2, 3, 5, 11]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'other', 'foo', 'world'],
                    'value': [50, 60, 70, 80, 90]})
df1

,lkey,value
0,foo,1
1,bar,2
2,other,3
3,foo,5
4,hello,11


In [99]:
df2

,rkey,value
0,foo,50
1,bar,60
2,other,70
3,foo,80
4,world,90


In [100]:
df1.merge(df2, left_on='lkey', right_on='rkey', suffixes=('_left', '_right'))

,lkey,value_left,rkey,value_right
0,foo,1,foo,50
1,foo,1,foo,80
2,foo,5,foo,50
3,foo,5,foo,80
4,bar,2,bar,60
5,other,3,other,70


---

---

In [101]:
nba_df = pd.read_csv("nba.csv")
cities_nba = pd.read_html("wikipedia_data.html")[1]
cities_nba = cities_nba.iloc[:-1, [0, 3, 5, 6, 7, 8]]

cities_nba.drop([16, 17, 19, 20, 21, 22, 23, 26, 29, 30, 31, 34, 35,
                 36, 37, 39, 40, 43, 44, 47, 48, 49, 50], inplace=True)


def proccess_regex(pattern, row):
    result = re.findall(pattern, row)
    if result:
        return result[0]
    return row


nba_df['team'] = nba_df['team'].apply(
    lambda x: proccess_regex('([a-zA-Z0-9\ ]+)(?:\*)?(?:\s)?(?:\(.*\))?', x)
)
nba_df = nba_df.head(30)

nba_df['team_ville'] = nba_df['team']
nba_df['team_ville'] = nba_df['team_ville'].map({
    'Toronto Raptors': 'Toronto',
    'Boston Celtics': 'Boston',
    'Philadelphia 76ers': 'Philadelphia',
    'Cleveland Cavaliers': 'Cleveland',
    'Indiana Pacers': 'Indianapolis',
    'Miami Heat': 'Miami–Fort Lauderdale',
    'Milwaukee Bucks': 'Milwaukee',
    'Washington Wizards': 'Washington, D.C.',
    'Detroit Pistons': 'Detroit',
    'Charlotte Hornets': 'Charlotte',
    'New York Knicks': 'New York City',
    'Brooklyn Nets': 'New York City',
    'Chicago Bulls': 'Chicago',
    'Orlando Magic': 'Orlando',
    'Atlanta Hawks': 'Atlanta',
    'Houston Rockets': 'Houston',
    'Golden State Warriors': 'San Francisco Bay Area',
    'Portland Trail Blazers': 'Portland',
    'Oklahoma City Thunder': 'Oklahoma City',
    'Utah Jazz': 'Salt Lake City',
    'New Orleans Pelicans': 'New Orleans',
    'San Antonio Spurs': 'San Antonio',
    'Minnesota Timberwolves': 'Minneapolis–Saint Paul',
    'Denver Nuggets': 'Denver',
    'Los Angeles Clippers': 'Los Angeles',
    'Los Angeles Lakers': 'Los Angeles',
    'Sacramento Kings': 'Sacramento',
    'Dallas Mavericks': 'Dallas–Fort Worth',
    'Memphis Grizzlies': 'Memphis',
    'Phoenix Suns': 'Phoenix'})

df_nba = pd.merge(nba_df, cities_nba, left_on="team_ville",
                  right_on="Metropolitan area")

df_nba['W/L%'] = pd.to_numeric(df_nba['W/L%'])
df_nba['W'] = pd.to_numeric(df_nba['W'])
df_nba['L'] = pd.to_numeric(df_nba['L'])
df_nba['Population (2016 est.)[8]'] = pd.to_numeric(
    df_nba['Population (2016 est.)[8]']
)
df_nba = df_nba[[
    'team', 'W', 'L', 'W/L%', 'Metropolitan area',
    'Population (2016 est.)[8]']]
df_nba['W/L'] = df_nba['W'] / (df_nba['L'] + df_nba['W'])
df_nba = df_nba.groupby('Metropolitan area').mean().reset_index()
population_by_region = df_nba['Population (2016 est.)[8]']
win_loss_by_region = df_nba['W/L']
stats.pearsonr(population_by_region, win_loss_by_region)[0]


-0.17657160252844614

In [102]:
mlb_df = pd.read_csv("mlb.csv")
cities_mlb = pd.read_html("wikipedia_data.html")[1]
cities_mlb = cities_mlb.iloc[:-1, [0, 3, 5, 6, 7, 8]]

cities_mlb.drop([24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
            38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], inplace=True)
mlb_df = mlb_df.head(30)

mlb_df['team_ville'] = mlb_df['team']
mlb_df['team_ville'] = mlb_df['team_ville'].map({
    'Boston Red Sox': 'Boston',
    'New York Yankees': 'New York City',
    'Tampa Bay Rays': 'Tampa Bay Area',
    'Toronto Blue Jays': 'Toronto',
    'Baltimore Orioles': 'Baltimore',
    'Cleveland Indians': 'Cleveland',
    'Minnesota Twins': 'Minneapolis–Saint Paul',
    'Detroit Tigers': 'Detroit',
    'Chicago White Sox': 'Chicago',
    'Kansas City Royals': 'Kansas City',
    'Houston Astros': 'Houston',
    'Oakland Athletics': 'San Francisco Bay Area',
    'Seattle Mariners': 'Seattle',
    'Los Angeles Angels': 'Los Angeles',
    'Texas Rangers': 'Dallas–Fort Worth',
    'Atlanta Braves': 'Atlanta',
    'Washington Nationals': 'Washington, D.C.',
    'Philadelphia Phillies': 'Philadelphia',
    'New York Mets': 'New York City',
    'Miami Marlins': 'Miami–Fort Lauderdale',
    'Milwaukee Brewers': 'Milwaukee',
    'Chicago Cubs': 'Chicago',
    'St. Louis Cardinals': 'St. Louis',
    'Pittsburgh Pirates': 'Pittsburgh',
    'Cincinnati Reds': 'Cincinnati',
    'Los Angeles Dodgers': 'Los Angeles',
    'Colorado Rockies': 'Denver',
    'Arizona Diamondbacks': 'Phoenix',
    'San Francisco Giants': 'San Francisco Bay Area',
    'San Diego Padres': 'San Diego'})

df_mlb = pd.merge(mlb_df, cities_mlb, left_on="team_ville",
               right_on="Metropolitan area")

df_mlb['W'] = pd.to_numeric(df_mlb['W'])
df_mlb['L'] = pd.to_numeric(df_mlb['L'])
df_mlb['Population (2016 est.)[8]'] = pd.to_numeric(
    df_mlb['Population (2016 est.)[8]'])
he = ['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']
df_mlb = df_mlb[he]
df_mlb['W/L'] = df_mlb['W'] / (df_mlb['L'] + df_mlb['W'])
df_mlb = df_mlb.groupby('Metropolitan area').mean().reset_index()


population_by_region = df_mlb['Population (2016 est.)[8]']
win_loss_by_region = df_mlb['W/L']

stats.pearsonr(population_by_region, win_loss_by_region)[0]


0.15027698302669307

In [103]:
nfl_df = pd.read_csv("nfl.csv")
cities_nfl = pd.read_html("wikipedia_data.html")[1]
cities_nfl = cities_nfl.iloc[:-1, [0, 3, 5, 6, 7, 8]]

nfl_df.drop([0, 5, 10, 15, 20, 25, 30, 35], inplace=True)

cities_nfl.drop([13, 22, 27, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
            40, 41, 42, 43, 45, 46, 47, 49, 50], inplace=True)

def proccess_regex(pattern, row):
    result = re.findall(pattern, row)
    if result:
        return result[0]
    return row


nfl_df['team'] = nfl_df['team'].apply(
    lambda x: proccess_regex('([a-zA-Z0-9\ ]+)(?:[\*\+]*)', x)
)
nfl_df = nfl_df.head(32)
nfl_df['team_ville'] = nfl_df['team']
nfl_df['team_ville'] = nfl_df['team_ville'].map({
    'New England Patriots': 'Boston',
    'Miami Dolphins': 'Miami–Fort Lauderdale',
    'Buffalo Bills': 'Buffalo',
    'New York Jets': 'New York City',
    'Baltimore Ravens': 'Baltimore',
    'Pittsburgh Steelers': 'Pittsburgh',
    'Cleveland Browns': 'Cleveland',
    'Cincinnati Bengals': 'Cincinnati',
    'Houston Texans': 'Houston',
    'Indianapolis Colts': 'Indianapolis',
    'Tennessee Titans': 'Nashville',
    'Jacksonville Jaguars': 'Jacksonville',
    'Kansas City Chiefs': 'Kansas City',
    'Los Angeles Chargers': 'Los Angeles',
    'Denver Broncos': 'Denver',
    'Oakland Raiders': 'San Francisco Bay Area',
    'Dallas Cowboys': 'Dallas–Fort Worth',
    'Philadelphia Eagles': 'Philadelphia',
    'Washington Redskins': 'Washington, D.C.',
    'New York Giants': 'New York City',
    'Chicago Bears': 'Chicago',
    'Minnesota Vikings': 'Minneapolis–Saint Paul',
    'Green Bay Packers': 'Green Bay',
    'Detroit Lions': 'Detroit',
    'New Orleans Saints': 'New Orleans',
    'Carolina Panthers': 'Charlotte',
    'Atlanta Falcons': 'Atlanta',
    'Tampa Bay Buccaneers': 'Tampa Bay Area',
    'Los Angeles Rams': 'Los Angeles',
    'Seattle Seahawks': 'Seattle',
    'San Francisco 49ers': 'San Francisco Bay Area',
    'Arizona Cardinals': 'Phoenix'})

df_nfl = pd.merge(nfl_df, cities_nfl, left_on="team_ville",
               right_on="Metropolitan area")


df_nfl['W'] = pd.to_numeric(df_nfl['W'])
df_nfl['L'] = pd.to_numeric(df_nfl['L'])
df_nfl['Population (2016 est.)[8]'] = pd.to_numeric(
    df_nfl['Population (2016 est.)[8]'])
df_nfl = df_nfl[['team', 'W', 'L', 'Metropolitan area', 'Population (2016 est.)[8]']]
df_nfl['W/L'] = df_nfl['W'] / (df_nfl['L'] + df_nfl['W'])
df_nfl = df_nfl.groupby('Metropolitan area').mean().reset_index()

population_by_region = df_nfl['Population (2016 est.)[8]']
win_loss_by_region = df_nfl['W/L']

stats.pearsonr(population_by_region, win_loss_by_region)[0]


0.004922112149349409

In [104]:

nfl = df_nfl.copy()
nba = df_nba.copy()
mlb = df_mlb.copy()
nhl = df_nhl.copy()

nba_nfl = pd.merge(nba, nfl, on='Metropolitan area')
pval_nba_nfl = stats.ttest_rel(nba_nfl['W/L_x'], nba_nfl['W/L_y'])[1]
nba_nhl = pd.merge(nba, nhl, on='Metropolitan area')
pval_nba_nhl = stats.ttest_rel(nba_nhl['W/L_x'], nba_nhl['W/L_y'])[1]
mlb_nfl = pd.merge(mlb, nfl, on='Metropolitan area')
pval_mlb_nfl = stats.ttest_rel(mlb_nfl['W/L_x'], mlb_nfl['W/L_y'])[1]
mlb_nhl = pd.merge(mlb, nhl, on='Metropolitan area')
pval_mlb_nhl = stats.ttest_rel(mlb_nhl['W/L_x'], mlb_nhl['W/L_y'])[1]
mlb_nba = pd.merge(mlb, nba, on='Metropolitan area')
pval_mlb_nba = stats.ttest_rel(mlb_nba['W/L_x'], mlb_nba['W/L_y'])[1]
nhl_nfl = pd.merge(nhl, nfl, on='Metropolitan area')
pval_nhl_nfl = stats.ttest_rel(nhl_nfl['W/L_x'], nhl_nfl['W/L_y'])[1]

pv = {'NFL': {"NFL": np.nan, 'NBA': pval_nba_nfl, 'NHL': pval_nhl_nfl, 'MLB': pval_mlb_nfl},
      'NBA': {"NFL": pval_nba_nfl, 'NBA': np.nan, 'NHL': pval_nba_nhl, 'MLB': pval_mlb_nba},
      'NHL': {"NFL": pval_nhl_nfl, 'NBA': pval_nba_nhl, 'NHL': np.nan, 'MLB': pval_mlb_nhl},
      'MLB': {"NFL": pval_mlb_nfl, 'NBA': pval_mlb_nba, 'NHL': pval_mlb_nhl, 'MLB': np.nan}
      }


# Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
# df.loc["NFL","NFL"] should return np.nan
#sports = ['NFL', 'NBA', 'NHL', 'MLB']
#p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
p_values = pd.DataFrame(pv)
p_values


,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
